Pierre Navaro - [Institut de Recherche Mathématique de Rennes](https://irmar.univ-rennes1.fr) - [CNRS](http://www.cnrs.fr/)

# References
* [multiprocessing basics](https://pymotw.com/2/multiprocessing/basics.html)
* [Python 201: A multiprocessing tutorial](https://www.blog.pythonlibrary.org/2016/08/02/python-201-a-multiprocessing-tutorial/)

# Multiprocessing

- The multiprocessing allows the programmer to fully leverage multiple processors. 
- It runs on both Unix and Windows.
- The `Pool` object parallelizes the execution of a function across multiple input values.
- The if `__name__ == '__main__'` part is necessary.

In [16]:
from multiprocessing import Pool

def f(x): return x*x+1  # Function executed on worker processes.

if __name__ == '__main__': # Executed only on main process.
    with Pool(4) as p:
        print(p.map(f, list(range(8))))

[1, 2, 5, 10, 17, 26, 37, 50]


# The Process class

In multiprocessing, processes are spawned by creating a Process object and then calling its start() method. 

In [17]:
from multiprocessing import Process

def f(name):
    print('hello', name)

if __name__ == '__main__':
    p = Process(target=f, args=('bob',))
    p.start()
    p.join()

hello bob


# Contexts and start methods
Depending on the platform, multiprocessing supports 
three ways to start a process:
- *spawn*: The parent process starts a fresh python interpreter process. Unix and Windows. The default on Windows.
- *fork*: The child process, when it begins, is effectively identical to the parent process. All resources of the parent are inherited by the child process. Available on Unix only. The default on Unix.
- *forkserver*: A server process is started. When a new process is needed, the parent process connects to the server and requests that it fork a new process. Available on Unix.

To select a start method you use the `set_start_method()` in the if `__name__ == '__main__'` clause of the main module. 


# Exchanging objects between processes

## Queues

In [18]:
from multiprocessing import Process, Queue

def f(q):
    q.put([42, None, 'hello'])

if __name__ == '__main__':
    q = Queue()
    p = Process(target=f, args=(q,))
    p.start()
    print(q.get())    # prints "[42, None, 'hello']"
    p.join()


[42, None, 'hello']


## Pipes
Pipe() returns two connection objects. Each connection object has send() and recv() methods

In [19]:
from multiprocessing import Process, Pipe

def f(conn):
    conn.send([42, None, 'hello'])
    conn.close()

if __name__ == '__main__':
    parent_conn, child_conn = Pipe()
    p = Process(target=f, args=(child_conn,))
    p.start()
    print(parent_conn.recv())   # prints "[42, None, 'hello']"
    p.join()

[42, None, 'hello']


# Synchronization between processes

In [20]:
from multiprocessing import Process, Lock

def f(l, i):
    l.acquire()
    try:
        print( i, end = ' ')
    finally:
        l.release()

if __name__ == '__main__':
    lock = Lock()

    for num in range(10):
        Process(target=f, args=(lock, num)).start()

0 1 2 3 4 5 6 7 8 9 

In [21]:
def f( i):
    print( i, end = ' ')

if __name__ == '__main__':
    for num in range(10):
        Process(target=f, args=(num,)).start()

0 1 2 3 4 5 6 7 8 9 

# Shared memory between processes

In [22]:
from multiprocessing import Process, Value, Array

def f(n, a):
    n.value = 3.1415927
    for i in range(len(a)):
        a[i] = -a[i]

if __name__ == '__main__':
    num = Value('d', 0.0)
    arr = Array('i', range(10))

    p = Process(target=f, args=(num, arr))
    p.start()
    p.join()

    print(num.value)
    print(arr[:])

3.1415927
[0, -1, -2, -3, -4, -5, -6, -7, -8, -9]


# Server process
Object returned by Manager() controls a server process which holds Python objects and allows other processes to manipulate them using proxies.

In [23]:
from multiprocessing import Process, Manager

def f(d, l):
    d[1] = '1'
    d['2'] = 2
    d[0.25] = None
    l.reverse()

if __name__ == '__main__':
    with Manager() as manager:
        d = manager.dict()
        l = manager.list(range(10))

        p = Process(target=f, args=(d, l))
        p.start()
        p.join()

        print(d)
        print(l)

{1: '1', '2': 2, 0.25: None}
[9, 8, 7, 6, 5, 4, 3, 2, 1, 0]


## Parallel Pi calculation

In [24]:
import time
import random
from multiprocessing import Pool, cpu_count

def compute_pi(n):
    count = 0
    for i in range(n):
        x=random.random()
        y=random.random()
        if x*x + y*y <= 1: count+=1
    return count

if __name__=='__main__':
    
    elapsed_time = time.time()
    np = 4
    assert ( np <= cpu_count())
    print ('Number of cores {}'.format(np))
    n = 10000000
    part_count=[n//np for i in range(np)]
    pool = Pool(processes=np)   
    count=pool.map(compute_pi, part_count)
    elapsed_time = time.time() - elapsed_time
    print ("Estimated value of Pi : {0:.8f}".format(4*sum(count)/n))  
    print ("Elapsed time : {0:.8f}".format(elapsed_time)) 

Number of cores 4
Estimated value of Pi : 3.14247680
Elapsed time : 0.90231705


# Joblib

[Joblib](http://pythonhosted.org/joblib/) provides a simple helper class to write parallel for loops using multiprocessing. 

In [25]:
def f(x):
    return x*x

[f(x) for x in range(10)]

[0, 1, 4, 9, 16, 25, 36, 49, 64, 81]

In [26]:
from joblib import Parallel, delayed
Parallel(n_jobs=2)(delayed(f)(x) for x in range(10))

[0, 1, 4, 9, 16, 25, 36, 49, 64, 81]

# Exercice

Write the program in which two processes send packets of information back and forth a 100 times and record the amount of time required. 

In [1]:
import os
 
from multiprocessing import Process, current_process
 
 
def doubler(number):
    """
    A doubling function that can be used by a process
    """
    result = number * 2
    proc_name = current_process().name
    print('{0} doubled to {1} by: {2}'.format(
        number, result, proc_name))
 
 
if __name__ == '__main__':
    numbers = [5, 10, 15, 20, 25]
    procs = []
    proc = Process(target=doubler, args=(5,))
 
    for index, number in enumerate(numbers):
        proc = Process(target=doubler, args=(number,))
        procs.append(proc)
        proc.start()
 
    proc = Process(target=doubler, name='Test', args=(2,))
    proc.start()
    procs.append(proc)
 
    for proc in procs:
        proc.join()

5 doubled to 10 by: Process-2
10 doubled to 20 by: Process-3
15 doubled to 30 by: Process-4
20 doubled to 40 by: Process-5
25 doubled to 50 by: Process-6
2 doubled to 4 by: Test
